# Using distilbert model 

## Import data

In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../raw_data/raw_train_data.csv', nrows=180000)

In [ ]:
import string
import re

def clean_text(text):
    #No whitespaces in beginning or end
    text = text.strip()
    #lowercase
    text= text.lower()
    #remove numbers
    text = re.sub(r'\b\d+\b', '', text)

    text = re.sub(rf"[{re.escape(string.punctuation)}]", '', text)

    return text

(3600000, 2)

In [ ]:
df['clean_text'] = df['text'].apply(clean_text)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", padding_side = "right")
tokenizer("My tokenizers and model must match")

/Users/johannesb/.pyenv/versions/3.10.6/envs/sentiscope/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df["text"]

first_sentence_tokenized = tokenizer(df["text"][0])

first_sentence_tokenized

{'input_ids': [101, 2026, 19204, 17629, 2015, 1998, 2944, 2442, 2674, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
from transformers import TFAutoModel
model = TFAutoModel.from_pretrained("prajjwal1/bert-tiny", from_pt = True)

In [ ]:
tokenized_tensors = tokenizer(df["clean_text"].tolist(), max_length=400, padding = "max_length", truncation = True, return_tensors="tf")

,label,text,clean_text
1581982,1,those looking for entertainment need not bothe...,those looking for entertainment need not bothe...
1568876,2,double trouble: this was a funny book. this is...,double trouble this was a funny book this is a...
3118450,1,Cracktastic Product: I replaced my old filter ...,cracktastic product i replaced my old filter h...
3131087,1,Not My Cup of Tea: I'd not realized that this ...,not my cup of tea id not realized that this wa...
1820244,1,I was a bit disapointed.: After reading so man...,i was a bit disapointed after reading so many ...


In [ ]:
tokenized_tensors["input_ids"].shape

In [ ]:
from transformers import TFAutoModelForSequenceClassification

tuning_model = TFAutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", from_pt = True)

2025-06-11 16:18:14.464171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
df['label'] = df['label'] -1

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
X_train = tokenized_tensors["input_ids"]
y_train = df["label"]

<tf.Tensor: shape=(3600, 400), dtype=int32, numpy=
array([[  101,  2216,  2559, ...,     0,     0,     0],
       [  101,  3313,  4390, ...,     0,     0,     0],
       [  101,  8579, 10230, ...,     0,     0,     0],
       ...,
       [  101,  6429, 12124, ...,     0,     0,     0],
       [  101,  4299,  2002, ...,     0,     0,     0],
       [  101,  9364,  1045, ...,     0,     0,     0]], dtype=int32)>

In [ ]:
tuning_model.compile(optimizer= "adam", metrics= "accuracy")
tuning_model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=5)

In [ ]:
tuning_model.save(f"bert_tiny_{len(df)}", save_format="tf")

Epoch 1/5


KeyboardInterrupt: 